# Appendix C: 
## Endogenous Beveridge elasticity
Paraphrased from Michaillat and Saez:

In the DMP model, the Beveridge elasticity depends on the unemployment rate. But we show here that this dependence is weak, and therefore we can use a formula for the efficient unemployment rate that does not depend on unemployment or vacancy rates.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('fivethirtyeight')

In [ ]:
import sys
sys.path.insert(0, '../')
import bug

## Read the data
Here, we read from the [excel file](https://github.com/pascalmichaillat/unemployment-gap/blob/main/code/data.xlsx) provided with the unemployment-gap matlab package.

The goal of this notebook is to re-create analysis and some figures from the Unemployment Gap paper, so that we can verify we are getting the *same* outputs. (*Sameness* allowing for some small differences between the two language implementations)

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Recession dates', header=1, 
                   usecols=['Peak month', 'Trough month'],).drop([0]).reset_index() 
starts =  pd.to_datetime(df['Peak month'])
ends = pd.to_datetime(df['Trough month'])

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Unemployment rate (percent)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
unempl_rate = pd.Series(data=df['Unemployment rate (percent)'].values,
                       index=dates, name='unempl_rate')

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Vacancy rate (thousands)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
vac_rate_proxy = pd.Series(data=df['Vacancy rate (thousands)'].values,
                       index=dates, name='vacancy_rate_proxy')

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Vacancy level (thousands)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
vacancy_level = pd.Series(data=df['Vacancy level (thousands)'].values,
                       index=dates, name='vacancy_level')

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Labor force level (thousands of persons)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
labor_level = pd.Series(data=df['Labor force level (thousands of persons)'].values,
                       index=dates, name='labor_force_level')

In [ ]:
vacancy_rate_2001 = vacancy_level/labor_level
vacancy_rate_splice = pd.concat([vac_rate_proxy.loc[:'2000-12'], vacancy_rate_2001.loc['2001-01':]*100])

In [ ]:
# create the quarterly rates and log of quarterly rates
u_q = unempl_rate.resample('Q').mean()/100
u_q = u_q.loc[:u_q.last_valid_index()]
log_u_q = np.log(u_q)

v_q = vacancy_rate_splice.resample('Q').mean()/100
v_q = v_q.loc[:v_q.last_valid_index()]
log_v_q = np.log(v_q)

### Matching elasticity
Call the function `bug.compute_beveridge_elasticity()` with arguments:
  * log of unemployment rate, required
  * log of vacancy rate, required
  

In [ ]:
bev = bug.compute_beveridge_elasticity(log_u_q, log_v_q, )

The function `bug.compute_beveridge_elasticity()` outputs 2 things:
  * The estimated Beveridge elasticity and 95% confidence intervals for that estimate, in the form of a pandas DataFrame
  * The estimated linear regression coefficients from the piece-wise linear fit of log unemployment vs log vacancy
  
For next computing the matching elasticity, we just need the Beveridge elasticity


Call the function `bug.compute_matching_elasticity()` with arguments:
  * unemployment rate, required
  * Beveridge elasticity, required


In [ ]:
eta = bug.compute_matching_elasticity(u_q, bev[0].E)

# FIGURE A4
## Matching elasticity in the United States, 1951–2019

In [ ]:
ax = eta.plot(color='blueviolet', linewidth=2, figsize=(9, 6), label='matching_elasticity')

plt.ylim(0, .6)
bug.format_plot(ax, recession_dates=[starts, ends], augment_legend=True)

plt.ylabel('Matching Elasticity', fontsize=12)
plt.title('Matching Elasticity', fontsize=14)

### Endogenous Bev elasticity
Call the function `bug.compute_efficient_unemployment()` with arguments:
  * unemployment rate, required
  * vacancy rate, required
  * Beveridge elasticity, required
  * Social value of nonwork (zeta); optional, default zeta=0.26
  * Recruiting cost (kappa); optional, default kappa=0.92
  
Call the function `bug.compute_separation_efficacy()` with arguments:
  * unemployment rate, required
  * Matching elasticity, required
  * Labor market tightness (theta)


In [ ]:
ustar = bug.compute_efficient_unemployment(u_q, v_q, bev[0].E)

In [ ]:
theta = v_q/u_q
lamomega = bug.compute_separation_efficacy(u_q, eta, theta)

Call the function `bug.compute_endogenous_efficiency()` with arguments:
  * Matching elasticity, required
  * separation-efficiency ratio (lambda/omega), required
  * Social value of nonwork (zeta); optional, default zeta=0.26
  * Recruiting cost (kappa); optional, default kappa=0.92

In [ ]:
# here we are illustrating how to use the optional arguments 
# (even though we are just using default values)

ustar_endo, theta_star = bug.compute_endogenous_efficiency(eta, lamomega, zeta=.26, kappa=.92)

# FIGURE A5
## US efficient unemployment rate with endogenous Beveridge elasticity

In [ ]:
ax= ustar.plot(color='magenta', linewidth=2, figsize=(9, 6), label="Exogenous")
ustar_endo.plot(color='darkorange', linewidth=2,  label="Endogenous")

plt.ylim(0, .06)
bug.format_plot(ax, recession_dates=[starts, ends], augment_legend=True)

plt.ylabel('Efficient unemployment rate', fontsize=12)
plt.title('Beveridge Elasticity', fontsize=14)

# Appendix D: 
## Hosios condition in the DMP model

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Unemployment level (thousands of persons)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
unempl_level = pd.Series(data=df['Unemployment level (thousands of persons)'].values,
                        index=dates, name='unempl_level')

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Short-term unemployment level (thousands of persons)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
ushort_level = pd.Series(data=df['Short-term unemployment level (thousands of persons)'].values, 
                        index=dates, name='short_term_unempl_level')

Call the function `bug.compute_job_finding_rate()` with arguments:
  * monthly unemployment level, required
  * monthly short-term unemployment level, required
  * True/False for whether to convert monthly to quarterly, optional; default is True
  * True/False for whether to adjust the short-term level begining at 1994, as in Shimer (2012), optional; default is True
  
Call the function `bug.compute_matching_efficacy()` with arguments:
  * job-finding rate, required
  * Labor-market tightness (theta), required
  * Matching elasticity (eta), required

In [ ]:
f = bug.compute_job_finding_rate(unempl_level, ushort_level, True,)

In [ ]:
omega = bug.compute_matching_efficacy(f, theta, eta)

# FIGURE A6 
## Matching efficacy in the United States, 1951–2019

In [ ]:
ax= omega.plot(color='darkred', linewidth=2, figsize=(9, 6), label='match_efficacy')

plt.ylim(0, 4)
bug.format_plot(ax, recession_dates=[starts, ends], augment_legend=True)

plt.ylabel('Matching Efficacy', fontsize=12)
plt.title('Matching Efficacy', fontsize=14)

### Job-separation rate
Call the function `bug.compute_job_separation_rate()` with arguments:
  * monthly unemployment level, required
  * monthly short-term unemployment level, required
  * monthly labor force level, required
  * True/False for whether to convert monthly to quarterly, optional; default is True
  * True/False for whether to adjust the short-term level begining at 1994, as in Shimer (2012), optional; default is True

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Labor force level (thousands of persons)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
labor_level = pd.Series(data=df['Labor force level (thousands of persons)'].values,
                       index=dates, name='labor_force_level')

In [ ]:
lamb = bug.compute_job_separation_rate(unempl_level, ushort_level, labor_level, True,)

Call the function `bug.compute_hosios_efficiency()` with arguments:
  * Matching elasticity (eta), required
  * job separation rate (lambda), required
  * Matching efficacy (omega), required
  * An initial value for the efficient unemployment rate (u0), required
  * discount rate; optional, default is r=0.012
  * Social value of nonwork (zeta); optional, default zeta=0.26
  * Recruiting cost (kappa); optional, default kappa=0.92  


In [ ]:
#% Calibrate discount rate as discussed in online appendix D
r = 0.012
u_hosios, th_hosios = bug.compute_hosios_efficiency(eta, lamb, omega, u0=u_q[0], r=r)

# FIGURE A7
## Hosiosian efficient unemployment rate in the United States, 1951–2019

In [ ]:
ax= u_hosios.plot(color='darkorange', linewidth=2, figsize=(9, 6), label="Hosios" )
ustar_endo.plot(color='magenta', linewidth=2,  label="Endogenous")

plt.ylim(0, .06)
bug.format_plot(ax, recession_dates=[starts, ends], augment_legend=True)

plt.ylabel('Efficient unemployment rate', fontsize=12)
plt.title('Efficient unemployment rate', fontsize=14)